In [59]:
cd './Downloads/mmdetection'

[Errno 2] No such file or directory: './Downloads/mmdetection'
/Users/ruixiao/Downloads/mmdetection


In [2]:
import pickle
import json
import os
import numpy as np
import mmcv
from mmcv.visualization.image import imshow_bboxes, imshow
from mmcv.image.io import imwrite

In [3]:
file = pickle.load(open('./result_ep1.pkl','rb'))

In [5]:
with open('./data/obj365/objects365_Tiny_val.json', encoding='utf-8') as f:
    line = f.readline()
    d = json.loads(line)
    path = d['images']
    ann = d['annotations']
    data_list = []
    data_id = []
    gt_bboxes = []
    gt_id = []
    for i in path:
        data_list.append(i.get('file_name'))
        data_id.append(i.get('id'))
    for j in ann:
        gt_bboxes.append(j.get('bbox'))
        gt_id.append(j.get('image_id'))
    f.close()

In [10]:
# 每个图片中gt的个数
gt_count = []
for c in data_id:
    gt_count.append(gt_id.count(c))

In [12]:
len(gt_count)

622

In [38]:
# 将gt的个数转化成累加和
gt_cumsum = np.cumsum(gt_count)

In [87]:
# 将 ann 字典中的 'image_id', 'bbox' 提取出来
gt_dir = []
for i in ann:
    names = {'image_id', 'bbox'}
    dir1 = {key: value for key, value in i.items() if key in names}
    gt_dir.append(dir1)

In [96]:
data_path = []
for i in data_list:
    list = os.path.join('./data/obj365/val',i) 
    data_path.append(list)

In [105]:
# 选择cls_score>0.7的proposal
test = []
for i in range(len(file)):
    bboxes = file[i]
    test1 = []
    for h in bboxes:
        if h[4]>0.7:
            test1.append(h)
    test.append(test1)

In [106]:
# 绘制proposal
for i in range(len(file)):
    # bboxes长度为2000
    for j in test:
        index = len(j)
    bboxes = file[i][:index]
    imshow_bboxes(data_path[i], bboxes, show=False, out_file='./proposal_0.7/'+str(i)+'.jpg')

In [104]:
# ground truth 的文件路径 (使用绘制好proposal的图片路径)
gt_path = []
for i in range(len(data_list)):
    list = './proposal_0.7/'+str(i)+'.jpg'
    gt_path.append(list)

In [107]:
# 绘制 ground truth（超出边界了，只能画出前621个 val 图像）
for i in range(len(file)):
    if i == 0:
        bboxes = gt_bboxes[:gt_cumsum[i]]
        imshow_bboxes(gt_path[i], np.array(bboxes), show=False, colors='red',out_file='./proposal_0.7gt/'+str(i)+'.jpg')
    else:
        bboxes = gt_bboxes[gt_cumsum[i]:gt_cumsum[i+1]]
        imshow_bboxes(gt_path[i], np.array(bboxes), show=False, colors='red',out_file='./proposal_0.7gt/'+str(i)+'.jpg')

IndexError: index 622 is out of bounds for axis 0 with size 622